In [216]:
import os
import sys
import shutil
import pandas
import numpy
import ipystata
import matplotlib.pyplot as plt

from ipystata.config import config_stata
config_stata('/home/kmongeon/research/stata14/stata-mp',force_batch=True)

from ipystata.config import config_syntax_higlight    
config_syntax_higlight(True)  

pandas.options.display.max_columns = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pandas.options.display.float_format = '{:.2f}'.format

In [217]:
pwd

'/home/kmongeon/research/exercise_physiology/scripts'

In [218]:
try: 
    shutil.rmtree('out')
    os.mkdir('out')
except:
    pass

In [219]:
%%stata --output dm

import delimited ./data/IzzyLongitudinalWorkingMarch2015.csv,  case(lower) clear 
order id session sequence
sort id sequence
save ./out/di, replace

import delimited ./data/torque.csv, case(lower) clear
drop if session==0
drop sequence
rename new_sequence sequence
order id session sequence
sort id  sequence
foreach var in ptiso pt60 pt240 {
replace `var' = . if `var'==-9999
}
save ./out/dt, replace

** Identify potentially data with errors
* Nota: July  2016
import delimited ./data/NegativeIDs.csv, case(lower) clear
keep nota id sequence 
keep if nota==1
save ./out/db1, replace

* Baraket: Nov 2016
import delimited ./data/data_examine_falk_nov2016.csv, case(lower) clear
keep falk id sequence
keep if falk==1
save ./out/db2, replace

** additional ptiso data
import delimited ./data/Spring_2013_knee_extension.csv, case(lower) clear
save ./out/dk, replace

* merge
use ./out/di, clear
sort id sequence
merge 1:1 id sequence using ./out/dt,  nogen
merge 1:1 id sequence using ./out/db1, nogen
merge 1:1 id sequence using ./out/db2, nogen
merge 1:1 id sequence using ./out/dk 
drop if _merge==2
drop _merge
count

save ./out/dm, replace


(70 vars, 427 obs)
(note: file ./out/di.dta not found)
file ./out/di.dta saved
(14 vars, 421 obs)
(40 observations deleted)(65 real changes made, 65 to missing)
(65 real changes made, 65 to missing)
(65 real changes made, 65 to missing)
(note: file ./out/dt.dta not found)
file ./out/dt.dta saved
. * Nota: July  2016(32 vars, 109 obs)
(59 observations deleted)
(note: file ./out/db1.dta not found)
file ./out/db1.dta saved
. import delimited ./data/data_examine_falk_nov2016.csv, case(lower) clear
(34 vars, 473 obs)
(460 observations deleted)
(note: file ./out/db2.dta not found)
file ./out/db2.dta saved
. import delimited ./data/Spring_2013_knee_extension.csv, case(lower) clear
(4 vars, 50 obs)
(note: file ./out/dk.dta not found)
file ./out/dk.dta saved
. use ./out/di, clear

    Result                           # of obs.
    -----------------------------------------
    not matched                            58
        from master                        52  
        from using           

In [220]:
%%stata --output dm

use ./out/dm

rename radius_sos rsos
rename tibial_sos tsos

rename grip_strength_best grip 
rename mat_offset_new matu

rename godin_pa godin
rename ntxcreat ntxc
rename caloricintake cint
renam calcium calc 
rename vitd vitd 
rename v48 vitd2
rename paq_total_score paq
rename totmvh mvh

rename forearm_length flength
rename thigh_length   tlength

global U id session sequence 
global Y rsos tsos
global X grip ptiso pt60 godin paq ntxc cint calc vitd vitd2 mvh
global Z matu age gender
global D nota falk

drop if matu>5
drop if id==517 & sequence==7

sort id sequence
by id: egen seq = seq()
global U id session sequence seq

order $U $D $Y $X $Z 
xtset id seq
count 
sort id seq
by id: egen trips = max(seq)
order trips, after(seq)

gen boy = 1 if gender==0
gen girl = 1 if gender==1


replace grip = . if nota==1
*replace rsos = . if nota==1
*replace tsos = . if falk==1
*replace ptiso = . if falk==1

keep id sequence session seq nota falk rsos tsos grip ptiso ntx mvh godin paq ///
    cint matu gender
sum *



  (all newnames==oldnames)
(7 observations deleted)
(1 observation deleted)
       panel variable:  id (unbalanced)
        time variable:  seq, 1 to 4
                delta:  1 unit
  425
(199 missing values generated)
(226 missing values generated)
(50 real changes made, 50 to missing)
. *replace tsos = . if falk==1. keep id sequence session seq nota falk rsos tsos grip ptiso ntx mvh godin paq ///
>     cint matu gender

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
          id |        425    326.1976    197.5498        100        583
     session |        425    1.430588    .4957421          1          2
    sequence |        425    4.197647    1.833518          1          7
         seq |        425    1.868235    .8776861          1          4
        nota |         50           1           0          1          1
-------------+------------------------------------------------------

In [221]:
dm['rsoss'] = (dm['rsos'] - dm['rsos'].mean()) / dm['rsos'].std()
dm['tsoss'] = (dm['tsos'] - dm['tsos'].mean()) / dm['tsos'].std()
dm['grips'] = (dm['grip'] - dm['grip'].mean()) / dm['grip'].std()
dm['torqs'] = (dm['ptiso'] - dm['ptiso'].mean()) / dm['ptiso'].std()

dm['ntxs'] = (dm['ntx'] - dm['ntx'].mean()) / dm['ntx'].std()
dm['mvhs'] = (dm['mvh'] - dm['mvh'].mean()) / dm['mvh'].std()
dm['gods'] = (dm['godin'] - dm['godin'].mean()) / dm['godin'].std()
dm['paqs'] = (dm['paq'] - dm['paq'].mean()) / dm['paq'].std()

dm['cints'] = (dm['cint'] - dm['cint'].mean()) / dm['cint'].std()
dm['matus'] = (dm['matu'] - dm['matu'].mean()) / dm['matu'].std()

dm.head(2)

,id,session,sequence,seq,nota,falk,rsos,tsos,grip,ptiso,godin,paq,cint,mvh,matu,gender,ntx,rsoss,tsoss,grips,torqs,ntxs,mvhs,gods,paqs,cints,matus
0,100,1,1,1,nan,nan,3828.00,3601.00,nan,113.93,37.00,2.59,1572.00,105.71,-1.67,0,1443.05,0.12,-0.79,nan,-0.19,-1.10,-0.01,-0.90,-0.84,-0.05,-0.40
1,100,1,3,2,nan,nan,3898.00,3629.00,27.00,136.02,48.00,3.17,2219.01,93.62,-0.71,0,3738.62,0.82,-0.53,0.47,0.22,-0.55,-0.30,-0.62,0.03,1.29,0.09


In [222]:
# dm.describe()

In [223]:
# %%stata --data dm 
# set processors 20

# save ./out/dm, replace

In [224]:
# dm.nota.value_counts()
# dm.falk.value_counts()


# Table 1. Summary statatisitcs

In [225]:
dt = dm.groupby('sequence')['rsos', 'tsos', 'grip', 'ptiso', 'ntx', 'matu', 'mvh', 'godin', 'cint', 'paq'].count()


In [226]:
# dm[['rsos', 'tsos', 'grip', 'ptiso', 'ntx', 'matu', 'mvh', 'godin', 'cint', 'paq']]

In [227]:
ds = dm.groupby('seq')['rsos', 'tsos', 'grip', 'ptiso'].mean()
dm.groupby('seq')['rsos', 'tsos', 'grip', 'ptiso', 'ntx'].count()

d0 = dm[dm['gender']==0].groupby('seq')['rsos', 'tsos', 'grip', 'ptiso'].mean()
d1 = dm[dm['gender']==1].groupby('seq')['rsos', 'tsos', 'grip', 'ptiso'].mean()

t1 = ds.append(d0)
t1 = t1.append(d1)
#t1.to_csv("./out/t1.csv")
t1

,rsos,tsos,grip,ptiso,ntx
seq,,,,,
1,173,174,113,140,170
2,139,139,122,110,130
3,86,85,78,60,82
4,17,17,16,0,4


,rsos,tsos,grip,ptiso
seq,,,,
1,3792.21,3654.82,21.14,111.62
2,3820.22,3687.64,22.87,127.48
3,3842.10,3729.59,26.21,146.71
4,3880.53,3800.12,30.97,nan
1,3791.63,3652.71,21.32,116.70
2,3809.36,3676.25,24.29,131.64
3,3822.49,3705.89,28.82,157.85
4,3842.88,3788.75,36.29,nan
1,3792.89,3657.30,20.94,106.24


In [228]:
ds.iloc[3]/ds.iloc[0]-1
d0.iloc[3]/d0.iloc[0]-1
d1.iloc[3]/d1.iloc[0]-1

rsos    0.02
tsos    0.04
grip    0.47
ptiso    nan
dtype: float64

rsos    0.01
tsos    0.04
grip    0.70
ptiso    nan
dtype: float64

rsos    0.03
tsos    0.04
grip    0.28
ptiso    nan
dtype: float64

In [229]:
# %%stata --data dm 
# set processors 20
# keep id sequence rsos tsos grip ptiso ntx matu mvh godin cint paq
# xtset id sequence

# xtsum *

# Figure 2. Impact of muscle strength on bone properties

In [230]:
%%stata --data dm 
set processors 20

sum *

xtset id seq

gsem M1[id] -> mvhs


    The maximum number of processors or cores being used is changed from 32 to
    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
       index |        425         212    122.8312          0        424
          id |        425    326.1976    197.5498        100        583
     session |        425    1.430588    .4957421          1          2
    sequence |        425    4.197647    1.833518          1          7
         seq |        425    1.868235    .8776861          1          4
-------------+---------------------------------------------------------
        nota |         50           1           0          1          1
        falk |         11           1           0          1          1
        rsos |        415    3815.552    100.9095       3542       4140
        tsos |        415     3687.08    109.0384       3367       4015
        grip |        329     23.4583    7.499687       

# Figure 3. Constrained specification results.

In [231]:
%%stata --data dm 
set processors 20

xtset id seq

gsem /// 
    (M1[id]  grips@a matus@b ntxs@c gods@d  -> rsoss, ) /// 
    (M1[id]  torqs@a matus@b ntxs@c mvhs@d  -> tsoss, ) /// 
    (N1[id] cints@n matus@m -> grips ,) ///
    (N1[id] cints@n matus@m -> torqs ,) ///
    , cov(e.rsoss*e.tsoss) cov(e.grips*e.torqs) ///
    cov(M1[id]*N1[id])





    The maximum number of processors or cores being used is changed from 32 to       panel variable:  id (unbalanced)
        time variable:  seq, 1 to 4
                delta:  1 unit
>     (M1[id]  grips@a matus@b ntxs@c gods@d  -> rsoss, ) /// 
>     (M1[id]  torqs@a matus@b ntxs@c mvhs@d  -> tsoss, ) /// 
>     (N1[id] cints@n matus@m -> grips ,) ///
>     (N1[id] cints@n matus@m -> torqs ,) ///
>     , cov(e.rsoss*e.tsoss) cov(e.grips*e.torqs) ///
>     cov(M1[id]*N1[id])

Fitting fixed-effects model:

Iteration 0:   log likelihood = -1320.0726  
Iteration 1:   log likelihood = -1287.4908  (backed up)
Iteration 2:   log likelihood =  -1266.641  
Iteration 3:   log likelihood = -1248.2717  
Iteration 4:   log likelihood = -1247.4659  
Iteration 5:   log likelihood = -1247.4654  
Iteration 6:   log likelihood = -1247.4654  

Refining starting values:

Grid node 0:   log likelihood = -1309.5834

Fitting full model:

Iteration 0:   log likelihood = -1309.5834  (not concave)
Iteration